In [ ]:
import itertools
import math
import time
from utils import *

MAX_POOL = 100          # newest 100 riders only
COST_PM  = 0.70         # $/mile driver cost

def pair_profit(r1, r2, cache):
    k = (r1.rider_id, r2.rider_id) if r1.rider_id < r2.rider_id else (r2.rider_id, r1.rider_id)
    if k not in cache:
        oi = (r1.pickup_lat,  r1.pickup_lon)
        di = (r1.dropoff_lat, r1.dropoff_lon)
        oj = (r2.pickup_lat,  r2.pickup_lon)
        dj = (r2.dropoff_lat, r2.dropoff_lon)
        trip, *_ = populate_shared_ride_lengths(oi, di, oj, dj)
        cache[k] = trip
    else:
        trip = cache[k]
    if trip == 0:
        return -math.inf
    rev  = r1.quoted_price*r1.solo_length + r2.quoted_price*r2.solo_length
    cost = COST_PM * trip
    return rev - cost

class StudentMatchingPolicy:
    # DO NOT MODIFY
    def __init__(self, c = 0.70):
        self.c = c

    # DO NOT MODIFY
    @staticmethod
    def get_name():
        return TEAM_NAME

    def matching_function(self, state, rider):
        """
        Returns the matched rider or None is there is no match

        Parameters
        ----------
        state: list
            list of rider(s) (object) waiting in the state
        rider: object
            An incoming rider

        Returns
        -------
        rider or None:
            Returns a matched rider (object) or None
        """
        """
        System‑aware greedy: choose waiting rider w that maximises
            gain = profit(rider,w) − best_alt_profit(w)
        No distance/profit cache is kept.
        """

        # ---- one‑time caches ----
        if not hasattr(self, "_dist_cache"):
            self._dist_cache = {}

        if not state:
            return None

        pool = state[-MAX_POOL:] if len(state) > MAX_POOL else state

        # 1) compute each w's best alternative profit
        best_alt = {w.rider_id: 0 for w in pool}
        for w1, w2 in itertools.combinations(pool, 2):
            pf = pair_profit(w1, w2, self._dist_cache)
            print(pf)
            if pf > best_alt[w1.rider_id]:
                best_alt[w1.rider_id] = pf
            if pf > best_alt[w2.rider_id]:
                best_alt[w2.rider_id] = pf

        # 2) evaluate net‑gain with incoming rider
        best_partner, best_gain = None, 0
        for w in pool:
            pair_pf = pair_profit(rider, w, self._dist_cache)
            gain    = pair_pf - best_alt[w.rider_id]
            if gain > best_gain:
                best_gain, best_partner = gain, w

        return best_partner if best_gain > 0 else None

## Test

In [ ]:
df = pd.read_csv("week_1_converted_with_state.csv")
state = {}
our_total_profit = 0
baseline_total_profit = 0
matched_pairs = set()

In [ ]:
# Simulate our model
for i, rider in df.iterrows():
    rider_id = rider['rider_id']
    if rider_id in matched_pairs:
        continue  # already matched

    # Try to match rider with someone in state
    best_match = None
    best_score = -np.inf

    for j_id, j_rider in state.items():
        features, profit, _ = compute_profit_features(rider, j_rider)
        if features:
            predicted_profit = model.predict([features])[0]
            if predicted_profit > best_score:
                best_score = predicted_profit
                best_match = j_id

    if best_match:
        j_rider = state.pop(best_match)
        _, profit, trip_len = compute_profit_features(rider, j_rider)
        our_total_profit += profit
        matched_pairs.update({rider_id, best_match})
    else:
        # no match, solo ride
        solo_profit = rider['quoted_price'] * rider['solo_length'] - c * rider['solo_length']
        our_total_profit += solo_profit
        state[rider_id] = rider

In [ ]:
# Simulate the example (baseline) profit
for i, rider in df.iterrows():
    if not pd.isna(rider['matching_outcome']):
        j = int(rider['matching_outcome'])
        if (rider['rider_id'], j) in matched_pairs or (j, rider['rider_id']) in matched_pairs:
            continue  # already counted
        j_rider = df[df['rider_id'] == j].iloc[0]
        _, profit, _ = compute_profit_features(rider, j_rider)
        if profit is not None:
            baseline_total_profit += profit
            matched_pairs.update({rider['rider_id'], j})
    else:
        solo_profit = rider['quoted_price'] * rider['solo_length'] - c * rider['solo_length']
        baseline_total_profit += solo_profit

In [ ]:
print(f"✅ Our matching model profit:     ${our_total_profit:.2f}")
print(f"🏁 Baseline example policy profit: ${baseline_total_profit:.2f}")

## Test 2

In [ ]:
cache = {}
MAX_POOL = 100

def matching_function(state, rider):
        """
        Returns the matched rider or None is there is no match

        Parameters
        ----------
        state: list
            list of rider(s) (object) waiting in the state
        rider: object
            An incoming rider

        Returns
        -------
        rider or None:
            Returns a matched rider (object) or None
        """
        """
        System‑aware greedy: choose waiting rider w that maximises
            gain = profit(rider,w) − best_alt_profit(w)
        No distance/profit cache is kept.
        """

        if not state:
            return None

        pool = state[-MAX_POOL:] if len(state) > MAX_POOL else state

        # 1) compute each w's best alternative profit
        best_alt = {w.rider_id: -0.5 for w in pool}
        for w1, w2 in itertools.combinations(pool, 2):
            pf = pair_profit(w1, w2, cache)
            print(pf)
            if pf > best_alt[w1.rider_id]:
                best_alt[w1.rider_id] = pf
            if pf > best_alt[w2.rider_id]:
                best_alt[w2.rider_id] = pf

        # 2) evaluate net‑gain with incoming rider
        best_partner, best_gain = None, 0
        for w in pool:
            pair_pf = pair_profit(rider, w, cache)
            gain    = pair_pf - best_alt[w.rider_id]
            if gain > best_gain:
                best_gain, best_partner = gain, w

        return best_partner if best_gain > 0 else None

In [ ]:
cache = {}
best_alt = {}
MAX_POOL = 180

def matching_function(state, rider):
    """
    O(n) system‑aware greedy; uses a persistent distance cache
    and updates best_alt incrementally.
    """
#     if not hasattr(self, "_dist_cache"):
#         self._dist_cache = {}
#         self._best_alt   = {}          # rider_id -> best alt profit

    if not state:
        return None

    # --- newest ≤ MAX_POOL riders ---
    pool = state[-MAX_POOL:] if len(state) > MAX_POOL else state

    # --- ensure best_alt entries exist ---
    for w in pool:
#         self._best_alt.setdefault(w.rider_id, 0)
        best_alt.setdefault(w.rider_id, 0)


    best_partner, best_gain = None, 0

    for w in pool:
        # small spatial filter (optional but fast): same pickup area
        # if w.pickup_area != rider.pickup_area:  continue

#         pf = pair_profit(rider, w, self._dist_cache)        
        pf = pair_profit(rider, w, cache)

        # update w's alternative if this new profit is better
        if pf > best_alt[w.rider_id]:
#             self._best_alt[w.rider_id] = pf
            best_alt[w.rider_id] = pf


#         gain = pf - self._best_alt[w.rider_id]
        gain = pf - best_alt[w.rider_id]
        if gain > best_gain:
            best_gain, best_partner = gain, w

    # register rider for future alternatives
    if best_gain <=0:
        best_alt[rider.rider_id] = 0
#         self._best_alt[rider.rider_id] = 0

    return best_partner if best_gain > 0 else None

## test for all

In [2]:
import pandas as pd, numpy as np, itertools, math, time
import matplotlib.pyplot as plt
from utils import populate_shared_ride_lengths                 # course helper

# --------------------------------------------------------------------------
# PARAMETERS
# --------------------------------------------------------------------------
MAX_POOL = 180          # newest riders seen by matcher
COST_PM  = 0.70
CSV_CONV = "data/week_1_converted_with_state.csv"
CSV_NON  = "data/week_1_convert_0.csv"

# --------------------------------------------------------------------------
# PRE‑TRAINED TABLES (load your real ones here)
# --------------------------------------------------------------------------
import glob
import collections

# Define price bins clearly
price_bins = [(0.48,0.54),(0.54,0.6),(0.6,0.65),(0.65,0.7),(0.7,0.75),(0.75,0.8)]

# Initialize alpha-beta distributions
alpha = collections.defaultdict(lambda: 1)
beta = collections.defaultdict(lambda: 1)

global_alpha = [1] * len(price_bins)
global_beta = [1] * len(price_bins)

agg_alpha_by_pickup = collections.defaultdict(lambda: [1]*len(price_bins))
agg_beta_by_pickup = collections.defaultdict(lambda: [1]*len(price_bins))

agg_alpha_by_dropoff = collections.defaultdict(lambda: [1]*len(price_bins))
agg_beta_by_dropoff = collections.defaultdict(lambda: [1]*len(price_bins))

def assign_arm(price):
    for idx, (low, high) in enumerate(price_bins):
        if low <= price < high:
            return idx
    return len(price_bins)-1

def discretize_state_size(size):
    return '0' if size==0 else '1-4' if size<=4 else '5-10' if size<=10 else '11+'

def discretize_avg_wait(wait):
    return '0-30' if wait<=30 else '31-100' if wait<=100 else '100+'

# Load weekly files separately
weeks_converted = sorted(glob.glob('data/week_*_converted_with_state.csv'))
weeks_non_converted = sorted(glob.glob('data/week_*_convert_0.csv'))

# iterate week by week explicitly
for week_idx, (conv_file, nonconv_file) in enumerate(zip(weeks_converted, weeks_non_converted), start=1):

    df_converted = pd.read_csv(conv_file)
    df_non_converted = pd.read_csv(nonconv_file)

    df_converted['convert_or_not'] = 1
    df_non_converted['convert_or_not'] = 0

    # Concatenate and sort this week's data explicitly
    df_week = pd.concat([df_converted, df_non_converted], ignore_index=True)
    df_week = df_week.sort_values(by='arrival_time').reset_index(drop=True)

    # Step 3: Explicitly RESET STATE EACH WEEK
    state = {}  # reset here explicitly (important)

    # Iterate clearly through riders of current week
    for idx, row in df_week.iterrows():
        t_now = row['arrival_time']  # continuous time is fine
        
        # Remove riders whose patience expired explicitly
        state = {rid: (arr, pat) for rid, (arr, pat) in state.items() if (t_now - arr) < pat}

        # Define context explicitly (with reset weekly state)
        context = (
            row['pickup_area'],
            row['dropoff_area'],
            discretize_state_size(len(state)),
            discretize_avg_wait(np.mean([t_now - arr for arr, _ in state.values()]) if state else 0)
        )

        # Update alpha-beta based on observed conversions clearly
        arm = assign_arm(row['quoted_price'])
        converted = row['convert_or_not']
        
        if converted:
            alpha[(context, arm)] += 1
            global_alpha[arm] += 1
            agg_alpha_by_pickup[row['pickup_area']][arm] +=1
            agg_alpha_by_dropoff[row['dropoff_area']][arm] +=1
            if row['waiting_time'] > 0:
                state[row['rider_id']] = (t_now, row['waiting_time'])
        else:
            beta[(context, arm)] += 1
            global_beta[arm] += 1
            agg_beta_by_pickup[row['pickup_area']][arm] +=1
            agg_beta_by_dropoff[row['dropoff_area']][arm] += 1

# --------------------------------------------------------------------------
# PRICING  (your generalized TS rule, unchanged)
# --------------------------------------------------------------------------
def pricing_policy_generalized(state_dict, pk, dp, t_now):
    state_size = discretize_state_size(len(state_dict))
    avg_wait   = discretize_avg_wait(
        np.mean([t_now-arr for arr,_ in state_dict.values()]) if state_dict else 0
    )
    ctx = (pk, dp, state_size, avg_wait)

    if any((ctx,a) in alpha for a in range(len(price_bins))):
        samples = [(np.random.beta(alpha[(ctx,a)], beta[(ctx,a)]), a)
                   for a in range(len(price_bins))]
        top2 = sorted(samples, reverse=True)[:2]
        return np.mean([np.mean(price_bins[a]) for _,a in top2])

    a_pick = agg_alpha_by_pickup.get(pk,  [1]*len(price_bins))
    b_pick = agg_beta_by_pickup .get(pk,  [1]*len(price_bins))
    a_drop = agg_alpha_by_dropoff.get(dp, [1]*len(price_bins))
    b_drop = agg_beta_by_dropoff .get(dp, [1]*len(price_bins))
    a_comb = [(x+y)/2 for x,y in zip(a_pick,a_drop)]
    b_comb = [(x+y)/2 for x,y in zip(b_pick,b_drop)]
    arm = np.argmax([np.random.beta(a,b) for a,b in zip(a_comb,b_comb)])
    return np.mean(price_bins[arm])

# --------------------------------------------------------------------------
# MATCHING  (O(n) net‑gain, uid‑based)
# --------------------------------------------------------------------------
MAX_POOL = 180 # 3-mins pairs
COST_PM = 0.7
BASE_PRICE_FALLBACK = 0.65
base_price = pd.read_csv('base_price_table.csv')
def pair_profit(r1, r2, cache):
    key = (r1._uid, r2._uid) if r1._uid < r2._uid else (r2._uid, r1._uid)
    if key not in cache:
        trip, *_ = populate_shared_ride_lengths(
            (r1.pickup_lat,  r1.pickup_lon), (r1.dropoff_lat, r1.dropoff_lon),
            (r2.pickup_lat,  r2.pickup_lon), (r2.dropoff_lat, r2.dropoff_lon)
        )
        cache[key] = trip
    else:
        trip = cache[key]

    if trip == 0:
        return -math.inf
    
    match1 = base_price.loc[(base_price["pickup_area"] == r1.pickup_area) &
                            (base_price["dropoff_area"] == r1.dropoff_area),
                            "base_price"]
    match2 = base_price.loc[(base_price["pickup_area"] == r2.pickup_area) &
                            (base_price["dropoff_area"] == r2.dropoff_area),
                            "base_price"]

    # if the pair exists, take the scalar; otherwise fall back (e.g. 0.65)
    quoted_price_1 = match1.iat[0] if len(match1) else BASE_PRICE_FALLBACK
    quoted_price_2 = match2.iat[0] if len(match2) else BASE_PRICE_FALLBACK

    rev  = quoted_price_1 * r1.solo_length + quoted_price_2 * r2.solo_length
    return rev - COST_PM * trip

def matching_function(state, rider, cache, best_alt):
    if not state: return None
    pool = state[-MAX_POOL:] if len(state)>MAX_POOL else state
    for w in pool: best_alt.setdefault(w._uid,-1.0)
    best_p, best_g = None, 0
    for w in pool:
        pf = pair_profit(rider,w,cache)
        gain = pf - best_alt[w._uid]
        if pf > best_alt[w._uid]:
            best_alt[w._uid] = pf
        if gain > best_g:
            best_g, best_p = gain, w
    best_alt[rider._uid] = 0
    return best_p if best_g>0 else None

# --------------------------------------------------------------------------
#  DATA PREP
# --------------------------------------------------------------------------
df_conv = pd.read_csv(CSV_CONV);  df_non = pd.read_csv(CSV_NON)
df_conv["convert_or_not"]=1; df_non["convert_or_not"]=0
df_conv["is_test_rider"]=False; df_non["is_test_rider"]=True
df_week = (pd.concat([df_conv,df_non])
             .sort_values("arrival_time")
             .reset_index(drop=True))
df_week["new_policy_price"]=np.nan

# --------------------------------------------------------------------------
#  SIMULATION
# --------------------------------------------------------------------------
state=[];  uid_seq=0
dist_cache = {};  best_alt = {}
profit=0.0; t_log=[]
match = 0
for _, row in df_week.iterrows():
    # give each row a lightweight object view
    rider = row.to_dict()                           # dict
    rider = pd.Series(rider)                        # still Series
    rider._uid = uid_seq; uid_seq += 1

    # clean expired waiters
    t_now = rider["arrival_time"]
    state = [w for w in state if (t_now - w.arrival_time) < w.waiting_time]

    # pricing for test riders
    if rider["is_test_rider"]:
        state_dict = {w._uid: (w.arrival_time, w.waiting_time) for w in state}
        price = pricing_policy_generalized(state_dict,
                                           rider.pickup_area,
                                           rider.dropoff_area,
                                           t_now)
        df_week.loc[_,"new_policy_price"]=price
    else:
        price = rider["quoted_price"]               # from csv
    rider.quoted_price = price

    # match
    t0=time.perf_counter()
    partner = matching_function(state, rider, dist_cache, best_alt)
    t_log.append(time.perf_counter()-t0)

    if partner is not None:
        state = [w for w in state if w._uid!=partner._uid]
        oi = (rider.pickup_lat,  rider.pickup_lon)
        di = (rider.dropoff_lat, rider.dropoff_lon)
        oj = (partner.pickup_lat,  partner.pickup_lon)
        dj = (partner.dropoff_lat, partner.dropoff_lon)
        trip_length, *_ = populate_shared_ride_lengths(oi, di, oj, dj)
        profit += (partner.quoted_price*partner.solo_length + rider.quoted_price*rider.solo_length - COST_PM * trip_length)
        match += 2
    elif rider["convert_or_not"]==1 and rider["waiting_time"]>0:
        # only converted riders can wait
        state.append(rider)

# solo riders left
for w in state:
    profit += w.quoted_price*w.solo_length - COST_PM*w.solo_length

print(f"✅ Total profit week‑1:  ${profit:,.2f}")
t = np.array(t_log)*1000
print(f"mean {t.mean():.2f} ms  95‑pct {np.percentile(t,95):.2f} ms  max {t.max():.2f} ms")

# --------------------------------------------------------------
#  simple price comparison
# --------------------------------------------------------------
test_rows = df_week[df_week.is_test_rider]
print("Original mean (rejected):", test_rows.quoted_price.mean())
print("New‑policy mean        :", test_rows.new_policy_price.mean())
lower = (test_rows.new_policy_price < test_rows.quoted_price).sum()
print(f"Lower price suggestions: {lower}/{len(test_rows)}  "
      f"({lower/len(test_rows):.1%})")

✅ Total profit week‑1:  $42.08
mean 3.11 ms  95‑pct 5.21 ms  max 7.79 ms
Original mean (rejected): 0.5995695574406348
New‑policy mean        : 0.6504690889370901
Lower price suggestions: 315/922  (34.2%)


In [3]:
match

340